In [2]:
import os
import torch
import pandas as pd
import evaluate
import numpy as np
import seaborn as sns
import time
import wandb
import warnings
import json
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoConfig, TrainingArguments, Trainer, pipeline
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from datetime import datetime
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

## Restaurant Search NER Recognition By Fine Tuning DistilBERT

## The Dataset
### MIT Restaurant Dataset

In [3]:
warnings.filterwarnings("ignore") #Don't do in production

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
train = pd.read_csv("Data/train.bio", sep="\t", header=None)
test = pd.read_csv("Data/test.bio", sep="\t", header=None)
train.head()

,0,1
0,B-Rating,2
1,I-Rating,start
2,O,restaurants
3,O,with
4,B-Amenity,inside
